In [1]:
import configparser
import json
import requests
import pandas as pd
import numpy as np
import random
from geopy.geocoders import Nominatim

# library for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# transform json file into a padas dataframe library
from pandas.io.json import json_normalize
import folium

In [2]:
config = configparser.ConfigParser()
config.read("./keys.ini")

api_key = config.get('foursquare', 'ApiKey')
api_secret = config.get('foursquare', 'ApiSecret')
access_token = config.get('foursquare', 'AccessToken')

## Find a coffee shop near a given location

In [4]:
url = "https://api.foursquare.com/v2/venues/explore"

In [5]:
params = dict (
client_id=api_key,
client_secret=api_secret,
v="20210623",
ll="28.66023, 77.14980",
query='coffee',
limit=3)

In [6]:
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [9]:
data.keys()

dict_keys(['meta', 'response'])

In [10]:
data['meta']

{'code': 200, 'requestId': '60d35024b8196d792a9374a3'}

In [12]:
data['response'].keys()

dict_keys(['suggestedFilters', 'warning', 'suggestedRadius', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [24]:
items = data['response']['groups'][0]['items']
len(items)

3

In [29]:
for item in items:
    print(f"[i] Name: {item['venue']['name']}")
    print(f"    Location: {item['venue']['location']['address']}")

[i] Name: Cafe Coffee Day
    Location: Moti Nagar
[i] Name: Dunkin'
    Location: Punjabi Bagh Club Road
[i] Name: Costa Coffee
    Location: 1st Floor, Moments Mall


## Conrad Hotel

Let's again assume that we are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.

In [3]:
address = "102 North End Ave, New York, NY"
geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f"Lat: {latitude}; Lon: {longitude}")

Lat: 40.7149555; Lon: -74.0153365


In [4]:
latlon = f"{latitude}, {longitude}"
latlon

'40.7149555, -74.0153365'

### Search for a specific venue category

Assume it's a lunch time and craving for italian food. Let's define a query to search for italian food that is within 500 metres from Conrad Hotel

In [5]:
url = "https://api.foursquare.com/v2/venues/search"
params = dict(
    client_id=api_key,
    client_secret=api_secret,
    v="20210623",
    ll=latlon,
    oauth_token=access_token,
    query="Italian",
    radius=500,
    limit=10
)

In [6]:
results = requests.get(url=url, params=params)

In [7]:
results

<Response [200]>

In [8]:
results = json.loads(results.text)

In [9]:
venues = results['response']['venues']
df = pd.json_normalize(venues)
df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1624539397,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta..."
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1624539397,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta..."


### define information of interest and filter dataframe

In [10]:
filtered_columns = ['id', 'name', 'categories']
filtered_columns.extend([col for col in df.columns if col.startswith('location')])
df_filtered = df.loc[:, filtered_columns]
df_filtered

,id,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta..."
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta..."


### visualize the map

In [11]:
restaurants = df_filtered.name
restaurants

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
Name: name, dtype: object

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add a red circle marker to represent Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(venues_map)

# add the italian restaurants as blue circle markers
for lat, long, label in zip(df_filtered['location.lat'], df_filtered['location.lng'], df_filtered.categories):
    folium.CircleMarker(
        [lat, long],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Explore a given venue

### Let's explore the closest Italian Restaurant -- Harry's Italian Pizza Bar

In [13]:
df_filtered

,id,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta..."
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta..."


In [14]:
venue_id = df_filtered.id.values[0]
venue_id

'4fa862b3e4b0ebff2f749f06'

In [32]:
url_venue = f"https://api.foursquare.com/v2/venues/{venue_id}"
params = dict(
    client_id=api_key,
    client_secret=api_secret,
    oauth_token=access_token,
    v='20210624'
)

results = requests.get(url=url_venue, params=params)

In [33]:
results

<Response [200]>

In [36]:
res_json = results.json()
res_json['response']['venue'].keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])

In [37]:
res_json['response']['venue']['rating']

6.3

In [38]:
res_json['response']['venue']['likes']

{'count': 121,
 'groups': [{'type': 'others', 'count': 121, 'items': []}],
 'summary': '121 Likes'}

In [39]:
res_json['response']['venue']['dislike']

False

In [40]:
res_json['response']['venue']['like']

False

### let's explore other one

In [41]:
venue_id = df_filtered.id.values[1]
url_venue2 = f"https://api.foursquare.com/v2/venues/{venue_id}"
params = dict(
    client_id=api_key,
    client_secret=api_secret,
    oauth_token=access_token,
    v='20210624'
)

results2 = requests.get(url=url_venue2, params=params)

In [42]:
results2

<Response [200]>

In [43]:
res2_json = results2.json()
ven2 = res2_json['response']['venue']

In [47]:
ven2.keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes'])

In [48]:
ven2['likes']

{'count': 1,
 'groups': [{'type': 'others',
   'count': 1,
   'items': [{'id': '587297293',
     'firstName': 'Dilshan',
     'lastName': 'Kanagaratnam',
     'gender': 'none',
     'countryCode': 'LK',
     'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
      'suffix': '/blank_boy.png',
      'default': True}}]}],
 'summary': 'Dilshan Kanagaratnam'}

In [49]:
ven2['tips']

{'count': 0, 'groups': []}

check tips of Harry's

In [50]:
harrys =  res_json['response']['venue']
harrys['tips']

{'count': 56,
 'groups': [{'type': 'following',
   'name': 'Tips from people you follow',
   'count': 0,
   'items': []},
  {'type': 'others',
   'name': 'All tips',
   'count': 56,
   'items': [{'id': '53d27909498e0523841340b6',
     'createdAt': 1406302473,
     'text': "Harry's Italian Pizza bar is known for it's amazing pizza, but did you know that the brunches here are amazing too? Try the Nutella French toast and we know you'll be sold.",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/53d27909498e0523841340b6',
     'likes': {'count': 4,
      'groups': [{'type': 'others', 'count': 4, 'items': []}],
      'summary': '4 likes'},
     'like': False,
     'logView': True,
     'agreeCount': 3,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '87473404',
      'firstName': 'TenantKing.com',
      'gender': 'none',
      'countryCode': 'US',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/87473404-HI5DTBT

#### Get Harry's tips

In [66]:
v_id = df_filtered.id.values[0]
url_tips = f"https://api.foursquare.com/v2/venues/{v_id}/tips"

params = dict(
    client_id=api_key,
    client_secret=api_secret,
    oauth_token=access_token,
    v='20210624',
    sort='recent',
    limit=20,
    offset=20
)
res_tips = requests.get(url=url_tips, params=params)

In [67]:
res_tips

<Response [200]>

In [70]:
#  count of tips
res_tips.json()['response']['tips']['count']

56

In [72]:
tips = res_tips.json()['response']['tips']['items'][0]

In [73]:
tips

{'id': '5cbda1f5febf31002cac0563',
 'createdAt': 1555931637,
 'text': 'Classic Square Pizza is the way to go here.',
 'type': 'user',
 'canonicalUrl': 'https://foursquare.com/item/5cbda1f5febf31002cac0563',
 'likes': {'count': 0, 'groups': []},
 'like': False,
 'logView': True,
 'agreeCount': 1,
 'disagreeCount': 0,
 'todo': {'count': 0},
 'user': {'id': '21563126',
  'firstName': 'Richard',
  'lastName': 'Revilla',
  'gender': 'male',
  'countryCode': 'US',
  'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
   'suffix': '/21563126_v05J1KPw_SVj6Ehq9g8B9jeAGjFUMsU5QGl-NZ8inUQ7pKQm5bKplW37EmR7jS2A7GYPBBAtl.jpg'}},
 'authorInteractionType': 'liked'}

In [65]:
tips

[{'id': '513b95ade4b0ef9359ef0cda',
  'createdAt': 1362859437,
  'text': 'Great spot! Has a legit bar with a ton of flat screens and plenty of room for drinking, socializing and pizza eating. The food is great. A must go in Financial District!',
  'type': 'user',
  'canonicalUrl': 'https://foursquare.com/item/513b95ade4b0ef9359ef0cda',
  'likes': {'count': 1,
   'groups': [{'type': 'others', 'count': 1, 'items': []}],
   'summary': '1 like'},
  'like': False,
  'logView': True,
  'agreeCount': 2,
  'disagreeCount': 0,
  'todo': {'count': 0},
  'user': {'id': '6343050',
   'firstName': 'Heather',
   'lastName': 'Couture',
   'gender': 'female',
   'countryCode': 'US',
   'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
    'suffix': '/CMRXTUXQAIZBQQEX.jpg'}}}]